In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch

from noteify.core.config import SAMPLE_RATE
from noteify.core.datasets import (MaestroDataset, MaestroDatasetProcessed, MusicAugmentor,
                                   MusicSegmentSampler, get_music_dataloader)
from noteify.core.evaluation import get_model_outputs
from noteify.core.models import TranscriptionNN
from noteify.core.utils import plot_audio, plot_roll
from noteify.utils import get_rel_pkg_path

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("Warning: Could not find GPU! Using CPU only")

In [ ]:
data_dir = get_rel_pkg_path("maestro/maestro-v3.0.0")
save_dir = get_rel_pkg_path("weights/")

In [ ]:
raw_dataset_test = MaestroDataset(data_dir, train=False, reduce_record_factor=10)

In [ ]:
batch_size = 16

In [ ]:
dataset_test = MaestroDatasetProcessed(raw_dataset_test)
sampler_test = MusicSegmentSampler(dataset_test, batch_size)
dataloader_test = get_music_dataloader(dataset_test, sampler_test, num_workers=None, pin_memory=True)

In [ ]:
model = TranscriptionNN()
model = model.to(device)
past_train_weights = os.path.abspath(os.path.join(save_dir, "Experiment 12-08-2020 05-15AM/Weights Best.pckl"))
model.load_state_dict(torch.load(past_train_weights))

In [ ]:
max_iter = 1
sampler_test.generate_segments()
result = get_model_outputs(model, dataloader_test, device, max_iter=max_iter,
                           return_inputs=True, return_targets=True, apply_sigmoid=True)
for n in range(max_iter):
    x = result['inputs'][n].numpy()
    frame_roll = result['targets']['frame_roll'][n].numpy()
    frame_outputs = result['outputs']['frame_output'][n].numpy()
    plot_audio(x)
    plot_roll(frame_roll)
    plot_roll(frame_outputs)

In [ ]:
%qtconsole